Se obtienen los datos de la db del sniim

In [1]:
import psycopg2
import numpy as np
import pandas as pd
from datetime import datetime

markets = [(1, [10, 17, 19, 23, 33, 40]), (42, [1])]

#Datos para obtener un mercado especifico
sourceid = 1
endid = 19
potatoid = 1

records = []

try:
    connection = psycopg2.connect(user = "postgres", password = "r351d3nc14501", host = "127.0.0.1", port = "5432", database = "potatoe_markets")
    cursor = connection.cursor()
    postgreSQL_select_Query = "SELECT P.sniimdatedate, P.avgprice FROM Price P INNER JOIN Market M ON P.SourceMarketID = M.MarketID INNER JOIN Market MM ON P.endmarketid = mm.marketid INNER JOIN Potato PP on p.potatoid = pp.potatoid WHERE m.marketid = {} AND MM.marketid = {} AND p.SNIIMPRESENTATION = 'CALCULADO' AND pp.potatoid = {} AND P.sniimdatedate > '2018-07-23' ORDER BY SNIIMDATEDATE".format(sourceid, endid, potatoid)
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from mobile table using cursor.fetchall")
    value_records = cursor.fetchall()

    print("Save each column value into the correspondent  array")
    for row in value_records:
        records.append(tuple((row[0], row[1])))
    
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


Selecting rows from mobile table using cursor.fetchall
Save each column value into the correspondent  array
PostgreSQL connection is closed


Se procesan los datos para rellenar fechas faltantes se guardan en la tabla datos procesados

In [2]:
dates = []
prices = []
for con in range(0, len(records)):
    dates.append(pd.Timestamp(records[con][0]))
    prices.append(records[con][1]/100)
dates = pd.Index(dates)
series = pd.Series(prices, dates)
df = pd.DataFrame({1:series})
#print(df)
df = df.asfreq('D')
df = df.reset_index()
df.columns = range(df.shape[1])

for index,row in df.iterrows():
    row[1] = pd.to_numeric(row[1], errors='coerce')
    if row[0].date().weekday() == 5 or row[0].date().weekday() == 6 :
        df = df.drop([index],axis=0)
        continue

df = df.interpolate()
print(df)

records_to_insert = []

for index,row in df.iterrows():
    records_to_insert.append(tuple((row[0], row[1], sourceid, endid, potatoid)))

#print(records_to_insert)

def bulkInsert(records):
    try:
        connection = psycopg2.connect(user = "postgres", password = "r351d3nc14501", host = "127.0.0.1", port = "5432", database = "potatoe_markets")
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO normalized_price (sniimdate, price, sourcemarketid, endmarketid, potatoid) VALUES (%s, %s, %s, %s, %s) """
        # executemany() to insert multiple rows rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into mobile table")
    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into mobile table {}".format(error))
    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
bulkInsert(records_to_insert)


#df.to_csv("potatoeseries04.csv",header=False, na_rep='NaN')

             0     1
0   2019-02-21  14.0
1   2019-02-22  15.0
4   2019-02-25  15.0
5   2019-02-26  15.0
6   2019-02-27  15.0
7   2019-02-28  15.0
8   2019-03-01  15.0
11  2019-03-04  15.0
12  2019-03-05  15.0
13  2019-03-06  15.0
14  2019-03-07  15.0
15  2019-03-08  15.0
18  2019-03-11  15.0
19  2019-03-12  15.0
20  2019-03-13  15.0
21  2019-03-14  15.0
22  2019-03-15  15.3
25  2019-03-18  15.3
26  2019-03-19  15.3
27  2019-03-20  15.3
28  2019-03-21  15.3
29  2019-03-22  15.3
32  2019-03-25  15.3
33  2019-03-26  15.3
34  2019-03-27  15.3
35  2019-03-28  15.3
36  2019-03-29  15.3
39  2019-04-01  15.3
40  2019-04-02  15.3
41  2019-04-03  15.3
..         ...   ...
160 2019-07-31  14.0
161 2019-08-01  14.0
162 2019-08-02  18.0
165 2019-08-05  18.0
166 2019-08-06  18.0
167 2019-08-07  18.0
168 2019-08-08  18.0
169 2019-08-09  18.0
172 2019-08-12  18.0
173 2019-08-13  15.5
174 2019-08-14  15.5
175 2019-08-15  15.5
176 2019-08-16  16.3
179 2019-08-19  16.3
180 2019-08-20  16.3
181 2019-08-2